In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb #for more visualizations

from scipy import stats #statistical tools

#import keras library to implement neural networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#import optimizers to train data
from tensorflow.keras.optimizers import Adam

#libraries to shuffle image data
import os #uses OS commands to change directories and make directories
import shutil #used to move image samples from directories
import random #used to shuffle samples
import glob

In [2]:
#sort dataset images by age (don't need to do here, but need to sort them when choosing whether to put image in train, test, or validate)
#need to be careful of face_scre values, need to modify Nan or weird excel format to inf on labels csv file
labeled = pd.read_csv("wiki_labels.csv") #reads in csv file to make sure it looks good and use to help sort images
judged = pd.read_csv("wiki_judge.csv")
#labeled.head(20)
#judged.head()

In [ ]:
#observe data set
plt.figure(figsize=(20,20))
sb.scatterplot(x='photo_taken', y='age', data=labeled)

In [4]:
#sort images into train, validate, and test datasets

#sources to help learn shutil and os libraries and processing image data
    #https://www.geeksforgeeks.org/os-module-python-examples/
    #https://deeplizard.com/learn/video/LhEMXbjGV_4

#create directories if it doesn't exist
#os.chdir()
#choose 8 different folders to combine into different directory and sort images for training, 2 for testing
parent_path = os.getcwd() + "\\wiki_labeled"
folders = [] #which folders are chosen (numbers b/w 00-99)

#make folders to sort random imgs to train and validate folders
os.chdir(parent_path)
os.mkdir("train")
os.mkdir("validate")
print(os.getcwd())

#get paths for train folder and enter into folder
os.chdir("train")
train_loc = os.getcwd()

#create category of ages to sort images (0-10, 11-20, etc.)
#need to fix the ages folders to be more specific? (1-79 folder for each age (79 folders), 80+ as another folder)
os.mkdir("0-20")
os.mkdir("21-40")
os.mkdir("41-60")
os.mkdir("61-80")
os.mkdir("81-100")
os.mkdir("101-120")
os.mkdir("121+")
# class_ages = []
# for i in range(1, 80):
#     os.mkdir(str(i))
#     class_ages.append(str(i))
# os.mkdir("80+")
# class_ages.append("80+")

#classes array
class_ages = ["0-20", "21-40", "41-60", "61-80", "81-100", "101-120", "121+"]

#print(train_loc)

num_folders = 20
num_valid = round(num_folders * 0.2)

#chose a number b/w 0-99, if b/w 0-9, need to format so that string name has 2 digits (ie. 00)
folders = random.sample(range(0, 100), num_folders) #train dataset
valid_f = [] #validate dataset

#separate 20% folders into the validate set 
for i in range(1, num_valid+1):
    valid_f.append(folders[num_folders-i])
    folders.pop(-1)

# print(folders)
# print(valid_f)

for i in folders:
    temp = ""
    if i >= 0 and i <= 9:
        temp = "0" + str(i)
    else:
        temp = str(i)
    #print(temp)
    #gets imgs in directory chosen (here is the path)
    img_folder = parent_path + "\\" + temp
    print(img_folder)
    #move files in these folders to the training folder
    #source: https://stackoverflow.com/questions/29685366/copy-files-from-multiple-directories-into-one-directory-using-python
    for img in glob.glob(img_folder + "\\*"): #any file with matching pathname will be copied to train folder
        #copy image to the training folder
        shutil.copy(img, train_loc)
        #print(img)
        
        #obtain path name of image
        parsedString = img.split("\\")
        imgToMatch = parsedString[-2] + "/" + parsedString[-1]
        #print(imgToMatch)
        
        #create directory here and add into age folder
        #source: https://davidhamann.de/2017/06/26/pandas-select-elements-by-string/
        matchedRow = labeled[labeled["full_path"].str.contains(imgToMatch)] #select rows that contain this partial string
        age = round(matchedRow["age"].values[0]) #rounded up age to make it easier to sort images
            #.values[0] obtains only the value you want for specific column versus obtaining the whoel series
        #print(matchedRow)
        #print(age)
        
        #obtain current directory for image (since it has been moved into the train folder)
        newImgLoc = train_loc + "\\" + parsedString[-1]
        
#         if os.path.isdir(age) is False: #if this age does not exist a folder in the train directory, create one
#             os.mkdir(age)
#             trainAge.append(age)
#         #obtain directory of age folder
#         ageLoc = train_loc + "\\" + age
#         #move image into respective age folder
#         shutil.move(newImgLoc, ageLoc)

        #sort image by age
        ageLoc = ""
        if age in range(0, 21):
            ageLoc = train_loc + "\\0-20"  
        elif age in range(21, 41):
            ageLoc = train_loc + "\\21-40"
        elif age in range(41, 61):
            ageLoc = train_loc + "\\41-60"
        elif age in range(61, 81):
            ageLoc = train_loc + "\\61-80"
        elif age in range(81, 101):
            ageLoc = train_loc + "\\81-100"
        elif age in range(101, 121):
            ageLoc = train_loc + "\\101-120"
        else:
            ageLoc = train_loc + "\\121+"
        
        #move image into respective age folder
        shutil.move(newImgLoc, ageLoc)    
        

#obtain path for validate folder and enter into folder
os.chdir("../")
os.chdir("validate")
valid_loc = os.getcwd()

#make folders for ages
os.mkdir("0-20")
os.mkdir("21-40")
os.mkdir("41-60")
os.mkdir("61-80")
os.mkdir("81-100")
os.mkdir("101-120")
os.mkdir("121+")

#move files into validate folder
for j in valid_f:
    temp = ""
    if j >= 0 and j <= 9:
        temp = "0" + str(j)
    else:
        temp = str(j)
    #gets imgs in directory chosen (here is the path)
    img_folder = parent_path + "\\" + temp
    print(img_folder)
    #move files in these folders to the training folder
    #source: https://stackoverflow.com/questions/29685366/copy-files-from-multiple-directories-into-one-directory-using-python
    for img in glob.glob(img_folder + "\\*"): #any file with matching pathname will be copied to train folder
        #print(img)
        #copy image to validate folder
        shutil.copy(img, valid_loc)
        
        #obtain path name of image
        parsedString = img.split("\\")
        imgToMatch = parsedString[-2] + "/" + parsedString[-1]
        
        #create directory here and add into age folder
        matchedRow = labeled[labeled["full_path"].str.contains(imgToMatch)] #select rows that contain this partial string
        age = round(matchedRow["age"].values[0]) #rounded up age to make it easier to sort images
            #.values[0] obtains only the value you want for specific column versus obtaining the whoel series
        
        #obtain current directory for image (since it has been moved into the train folder)
        newImgLoc = valid_loc + "\\" + parsedString[-1]
        
#         if os.path.isdir(age) is False: #if this age does not exist a folder in the train directory, create one
#             os.mkdir(age)
#             validAge.append(age)  
#         #obtain directory of age folder
#         ageLoc = valid_loc + "\\" + age
#         #move image into respective age folder
#         shutil.move(newImgLoc, ageLoc)

        #sort image by age
        ageLoc = ""
        if age in range(0, 21):
            ageLoc = valid_loc + "\\0-20"  
        elif age in range(21, 41):
            ageLoc = valid_loc + "\\21-40"
        elif age in range(41, 61):
            ageLoc = valid_loc + "\\41-60"
        elif age in range(61, 81):
            ageLoc = valid_loc + "\\61-80"
        elif age in range(81, 101):
            ageLoc = valid_loc + "\\81-100"
        elif age in range(101, 121):
            ageLoc = valid_loc + "\\101-120"
        else:
            ageLoc = valid_loc + "\\121+"
        
        #move image into respective age folder
        shutil.move(newImgLoc, ageLoc)

#return to wiki_labeled directory
os.chdir("../")
        

C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\01
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\78
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\94
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\08
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\57
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\84
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\37
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\55
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\09
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\23
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\11
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\95
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\34
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\59
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\17
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\85
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\58
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\68
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\91
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labele

In [3]:
#obtain test data set path
# os.chdir("wiki_labeled")
# os.chdir("train")
# train_loc = os.getcwd()
# os.chdir("../")
# os.chdir("validate")
# valid_loc = os.getcwd()
# os.chdir("../")
print(train_loc)
print(valid_loc)
os.chdir("../")
print(os.getcwd())

os.chdir("wiki_judge_images")
test_loc = os.getcwd()
print(test_loc)

class_ages = ["0-20", "21-40", "41-60", "61-80", "81-100", "101-120", "121+"]

C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\train
C:\Users\truon\CSCI 4931-DL\PA2\wiki_labeled\validate
C:\Users\truon\CSCI 4931-DL\PA2
C:\Users\truon\CSCI 4931-DL\PA2\wiki_judge_images


In [4]:
#put test dataset into a single class folder (can call it unknown or any other name)
unknown_path = ""
if os.path.isdir("unknown") is False:
    os.mkdir("unknown")
    unknown_path = test_loc + "\\unknown"

    for img in glob.glob(test_loc + "\\*"): #move test images to unknown folder
        shutil.move(img, unknown_path)
os.chdir("../")

In [5]:
#convert image batch so it can be processed in model
train_batch = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=train_loc, target_size=(100, 100), classes=class_ages, batch_size=20, shuffle=True)
validate_batch = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=valid_loc, target_size=(100, 100), classes=class_ages, batch_size=20, shuffle=True)
test_batch = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=test_loc, target_size=(100, 100), classes=None, class_mode=None, batch_size=1, shuffle=False)

Found 9949 images belonging to 7 classes.
Found 2426 images belonging to 7 classes.
Found 2001 images belonging to 1 classes.


In [7]:
#create CNN model here
#FIX: Make this CNN better in terms of RMSE
#add more layers, add more epochs, etc.
#added regularizers to help with overfitting problem
CNN = Sequential([
    Conv2D(filters = 45, kernel_size = (5, 5), padding = 'same', input_shape = (100, 100, 3)),
    BatchNormalization(axis = -1, epsilon = 0.001),
    Activation('relu'),
    Conv2D(filters = 45, kernel_size = (3, 3), kernel_regularizer = regularizers.l1(1e-4), activity_regularizer = regularizers.l2(1e-5), padding = 'same'),
    BatchNormalization(axis = -1, epsilon = 0.001),
    Activation('relu'),
    MaxPool2D(pool_size = (2,2), strides = 1),
    Dropout(0.2),
    
    Conv2D(filters = 30, kernel_size = (5, 5), padding = 'same'),
    BatchNormalization(axis = -1, epsilon = 0.001),
    Activation('relu'),
    Conv2D(filters = 30, kernel_size = (3, 3), kernel_regularizer = regularizers.l1(1e-4), activity_regularizer = regularizers.l2(1e-5), padding = 'same'),
    BatchNormalization(axis = -1, epsilon = 0.001),
    Activation('relu'),
    MaxPool2D(pool_size = (2,2), strides = 1),
    Dropout(0.2),
    
#     Conv2D(filters = 30, kernel_size = (5, 5), padding = 'same'),
#     BatchNormalization(axis = -1, epsilon = 0.001),
#     Activation('relu'),
#     Conv2D(filters = 15, kernel_size = (3, 3), kernel_regularizer = regularizers.l1(1e-4), activity_regularizer = regularizers.l2(1e-5), padding = 'same'),
#     BatchNormalization(axis = -1, epsilon = 0.001),
#     Activation('relu'),
#     MaxPool2D(pool_size = (2, 2), strides = 1),
#     Dropout(0.3),
    
    Flatten(),
#     Dense(units = 7, activation = 'relu'),
    Dense(units = 7, activation = 'softmax'), 
])
CNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 100, 100, 45)      3420      
_________________________________________________________________
batch_normalization_4 (Batch (None, 100, 100, 45)      180       
_________________________________________________________________
activation_4 (Activation)    (None, 100, 100, 45)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 100, 100, 45)      18270     
_________________________________________________________________
batch_normalization_5 (Batch (None, 100, 100, 45)      180       
_________________________________________________________________
activation_5 (Activation)    (None, 100, 100, 45)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 99, 99, 45)       

In [8]:
#compile model here
CNN.compile(optimizer = Adam(learning_rate=0.0001), loss = "categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.RootMeanSquaredError()])

In [9]:
#fit training set here
CNN.fit(x=train_batch, steps_per_epoch=50, epochs=20, validation_data=validate_batch, validation_steps=10, verbose=2)

Epoch 1/20
50/50 - 152s - loss: 10.5638 - accuracy: 0.4290 - root_mean_squared_error: 0.3815 - val_loss: 6.2833 - val_accuracy: 0.5450 - val_root_mean_squared_error: 0.3079
Epoch 2/20
50/50 - 232s - loss: 4.6849 - accuracy: 0.4660 - root_mean_squared_error: 0.3486 - val_loss: 2.9736 - val_accuracy: 0.4600 - val_root_mean_squared_error: 0.3114
Epoch 3/20
50/50 - 228s - loss: 4.4245 - accuracy: 0.4650 - root_mean_squared_error: 0.3524 - val_loss: 2.8661 - val_accuracy: 0.4100 - val_root_mean_squared_error: 0.3430
Epoch 4/20
50/50 - 229s - loss: 3.7637 - accuracy: 0.5040 - root_mean_squared_error: 0.3371 - val_loss: 2.9348 - val_accuracy: 0.2700 - val_root_mean_squared_error: 0.3821
Epoch 5/20
50/50 - 250s - loss: 3.2759 - accuracy: 0.4900 - root_mean_squared_error: 0.3377 - val_loss: 2.3706 - val_accuracy: 0.4800 - val_root_mean_squared_error: 0.3143
Epoch 6/20
50/50 - 331s - loss: 3.5005 - accuracy: 0.5030 - root_mean_squared_error: 0.3378 - val_loss: 2.5272 - val_accuracy: 0.2450 - val

In [10]:
#predict judged dataset
output = CNN.predict(test_batch, steps=len(test_batch), verbose=0)

In [23]:
print(output[0])
print(output[1])
print(output[2])
print(output[200])
largest = output.argmax(axis=-1)

[1.4398384e-04 3.1811437e-01 1.3702102e-01 5.4469866e-01 1.6519717e-06
 2.0107389e-07 2.0126881e-05]
[7.3714510e-02 5.3906274e-01 2.7701309e-01 8.9777686e-02 4.7228993e-03
 5.3238089e-04 1.5176699e-02]
[2.0096570e-04 8.6275834e-01 1.1762378e-01 1.9316586e-02 2.3232253e-05
 1.2727711e-05 6.4338994e-05]
[2.1592426e-01 5.2323115e-01 2.4185899e-01 1.8669952e-02 2.9179535e-04
 2.2489770e-07 2.3647157e-05]
3
1
1
1
2
2
1
3
1
3
2
1
1
2
2
0
1
2
1
1
1
1
1
1
2
1
3
3
1
1
3
1
2
1
0
3
2
1
2
1
1
3
1
1
2
1
1
1
1
2
1
3
1
1
2
1
1
2
2
2
2
3
1
1
1
3
1
2
1
1
3
1
1
1
1
1
3
2
1
1
2
2
1
1
1
1
1
1
1
1
1
2
2
0
1
2
1
1
1
1
2
1
1
1
3
1
2
3
2
2
1
1
1
1
1
0
2
1
1
3
2
2
1
2
2
1
2
1
2
2
3
1
2
3
2
1
1
1
1
1
1
1
1
3
1
1
1
1
1
2
2
2
1
1
2
1
1
1
3
1
1
1
1
1
1
2
1
1
1
1
2
1
1
2
2
1
1
1
2
2
1
1
1
2
1
1
1
2
1
1
0
2
1
2
2
1
2
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
2
1
1
1
2
1
3
3
1
1
2
2
2
2
1
2
1
1
2
3
1
1
3
1
3
2
1
1
1
1
1
2
2
1
1
3
2
2
2
1
1
1
1
2
1
1
1
1
2
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
2
2
1
1
1
1
1
1
3
2
1
2
2
1
1
1
2


In [21]:
#print output to file
outfile = open("judge_pred.csv", "w")
outfile.write("ID, Age\n")
for i, ages in enumerate(largest):
    outfile.write(str(i))
    outfile.write(", ")
    #find index of largest value in list and record age range in file
    if ages == 0:
        outfile.write("0-20")
    elif ages == 1:
        outfile.write("21-40")
    elif ages == 2:
        outfile.write("41-60")
    elif ages == 3:
        outfile.write("61-80")
    elif ages == 4:
        outfile.write("81-100")
    elif ages == 5:
        outfile.write("101-120")
    elif ages == 6:
        outfile.write("121+")
    outfile.write("\n")
outfile.close()

In [3]:
#delete folders after done training 
# os.chdir("../")
parent_path = os.getcwd() + "\\wiki_labeled"
os.chdir(parent_path)
# os.getcwd()
shutil.rmtree("train")
shutil.rmtree("validate")
# os.rmdir("train")
# os.rmdir("validate")
os.chdir("../")
#print(os.getcwd())